About Dataset
Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration

- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

### BEst Practices

1. Preprocessing And cleaning
2. Train, test split
3. BOW, TF-IDF, WOrd2Vec
4. Train ML Algorithms

In [1]:
import pandas as pd
data=pd.read_csv(r"E:\learning\krishudemy\nlpml\kindle\all_kindle_review.csv")

In [2]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
data.shape

(12000, 11)

In [4]:
df=data[["reviewText", "rating"]]

In [5]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [6]:
df.shape

(12000, 2)

In [7]:
# missing values

df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [8]:
# unique ratings

df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [9]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing and cleaning

In [10]:
## positive review is 1 and negative revies is 0

df['rating']=df['rating'].apply(lambda x: 0 if x<3 else 1) 

C:\Users\krish\AppData\Local\Temp\ipykernel_7180\4182422336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x: 0 if x<3 else 1)


In [11]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [12]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [13]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [14]:
## lowering all the cases

df['reviewText']=df['reviewText'].str.lower()

C:\Users\krish\AppData\Local\Temp\ipykernel_7180\800268701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [15]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [16]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from bs4 import BeautifulSoup
import re       

In [18]:
## removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+', '', x))

## removing the stopwords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

## removing url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

## removing html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

## removing any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\krish\AppData\Local\Temp\ipykernel_7180\2172794346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+', '', x))
C:\Users\krish\AppData\Local\Temp\ipykernel_7180\2172794346.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
C:\Users\krish\AppData\Local\Temp\ipykernel_7180\2172794346.py:8: SettingWithCopyWa

In [19]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [20]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [21]:
lemmatizer=WordNetLemmatizer()

In [22]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [23]:
df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))

C:\Users\krish\AppData\Local\Temp\ipykernel_7180\3266273754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))


In [24]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


## Train, Test Split

In [25]:
## Train, Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df['reviewText'], df['rating'], test_size=0.20)

## Applyin BOW

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()

In [27]:
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

## Applying TF-IDF

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [29]:
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [30]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
X_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Train the Model

In [32]:
from sklearn.ensemble import RandomForestClassifier
rfc_model_bow=RandomForestClassifier().fit(X_train_bow, y_train)
rfc_model_bow=RandomForestClassifier().fit(X_train_tfidf, y_train)

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [34]:
y_pred_bow=rfc_model_bow.predict(X_test_bow)

In [35]:
y_pred_tfidf=rfc_model_bow.predict(X_test_tfidf)

In [36]:
confusion_matrix(y_test,y_pred_bow)

array([[ 628,  201],
       [ 319, 1252]], dtype=int64)

In [46]:
print("BOW + Random Forest accuracy", accuracy_score(y_test, y_pred_bow))

BOW + Random Forest accuracy 0.7833333333333333


In [47]:
confusion_matrix(y_test,y_pred_tfidf)

array([[ 376,  453],
       [  59, 1512]], dtype=int64)

In [48]:
print("TF-IDF + Random Forest accuracy", accuracy_score(y_test, y_pred_tfidf))

TF-IDF + Random Forest accuracy 0.7866666666666666


## Applying Word2Vec

In [ ]:
import gensim.downloader as api
import numpy as np
from tqdm import tqdm
import re

In [42]:
w2v_model = api.load("word2vec-google-news-300")

In [59]:
df.shape

(12000, 2)

In [62]:
## to average word vectors for each review

def review_to_vector(review, model):
    """
    calculate the avg w2v vector for a doc (a single string)
    returns a zero vector if no words in the doc are in the models vocabulary
    """
    words=review.split()
    # get all word vectors for words that exists in the models vocabulary
    word_vectors=[model[word] for word in words if word in model]

    # if not words in the vocabulary return a zero vector of the correct size
    if not word_vectors:
        return np.zeros(model.vector_size)
    
    # or return the mean of the word vectors
    return np.mean(word_vectors, axis=0)

In [69]:
X_train_w2v = [review_to_vector(review, w2v_model) for review in tqdm(X_train)]

100%|██████████| 9600/9600 [00:01<00:00, 5799.94it/s]


In [70]:
X_test_w2v = [review_to_vector(review, w2v_model) for review in tqdm(X_test)]

100%|██████████| 2400/2400 [00:00<00:00, 9753.26it/s] 


In [71]:
X_train_w2v = np.array(X_train_w2v)
X_test_w2v = np.array(X_test_w2v)

In [72]:
rfc_model_w2v = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rfc_model_w2v.fit(X_train_w2v, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [73]:
y_pred_w2v = rfc_model_w2v.predict(X_test_w2v)

In [74]:
accuracy = accuracy_score(y_test, y_pred_w2v)
conf_matrix = confusion_matrix(y_test, y_pred_w2v)
class_report = classification_report(y_test, y_pred_w2v)

In [ ]:
print(f"\nWord2Vec + RandomForest accuracy: {accuracy:.4f}")

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)



Word2Vec + RandomForest Accuracy: 0.7696

Confusion Matrix:
[[ 387  442]
 [ 111 1460]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.47      0.58       829
           1       0.77      0.93      0.84      1571

    accuracy                           0.77      2400
   macro avg       0.77      0.70      0.71      2400
weighted avg       0.77      0.77      0.75      2400

